In [2]:
# import external libraries
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By 
from selenium.webdriver.chrome.service import Service as ChromeService 
from webdriver_manager.chrome import ChromeDriverManager 
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from tqdm import tqdm
from typing import *
import random
    
import warnings
warnings.filterwarnings('ignore')

# import local modules
from middlewares.scraper_functs import *

# Selenium configuration
service = ChromeService()
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)
# driver.implicitly_wait(10)
wait = WebDriverWait(driver, 5)
driver.maximize_window()

# Dictionary Definition

dataDict={
    'Company':[],
    'Company_Web_Site':[],
    'Category':[], 
    'Value':[],
    'Value_Cat':[],
    'Scope':[]
}

JobDict={
    'Index':[],
    'Company':[],
    'Company_Web_Site':[],
    'Title':[], 
    'job_details':[],
    'job_skills':[],
    'Job_Description':[],
    'Scope':[]
}

# Global Variables definition

unique_companies_source:int = None
unique_companies_after:int= None

# Url 
url_base = 'https://www.linkedin.com/jobs/search/?currentJobId=3736552023&geoId=102890719&keywords=esg&location=Netherlands&origin=JOB_SEARCH_PAGE_LOCATION_AUTOCOMPLETE&refresh=true'


# Definint the driver
driver.get(url_base)
# eliminate PopUp
eliminate_popup(driver,wait)
# login
login(driver)




In [3]:
try:
    index=None
    company=None
    ul_carrusel_len=len(driver.find_elements(By.XPATH,'//li[contains(@class,"artdeco-pagination__indicator artdeco-pagination__indicator--number")]'))
    

    if ul_carrusel_len > 0:
        for i in range(ul_carrusel_len):  
            driver.find_element(By.XPATH,f'//li[@data-test-pagination-page-btn="{i+1}"]').click()
            time.sleep(1*random.random())
            data_gatter_jobs(driver=driver,index=index,url=url_base,company=company,data_dict=JobDict)
    else:
        data_gatter_jobs(driver=driver,index=index,url=url_base,company=company,data_dict=JobDict)
    
except NoSuchElementException as e:
    

    JobDict['Company_Web_Site'].append(url_base)
    JobDict['Title'].append(None)
    JobDict['job_details'].append(None)
    JobDict['Job_Description'].append(None)
    JobDict['Scope'].append('Netherlands')  

In [4]:
df=pd.DataFrame(JobDict)

In [5]:
df.to_excel('data/ESG_jobs_3.xlsx',index=False)

In [ ]:

# Calculate Unique Source Companies
unique_companies_source=data_Source.Name.nunique()

# Execution Wrapper

def execution(function,
              working_dict:dict,
              webdriver,
              dataSource:pd.DataFrame) -> pd.DataFrame:
    
    """
    Summary:
    
    Wrapper to execute the linkedIn Elements
    
    agrs:
    
    function: function type that contain the main function to execute the exploration
    working_dict: dict type that holds the dict to be filled by the exploration job
    WebDriver: WebDriver type that holds the execution driver
    dataSource: Pandas Dataframe Type, contains the data to be iterate and explore
 
    """
    global unique_companies_after

    # Initialize the driver
    function(data_Source=dataSource,driver=webdriver,data_dict=working_dict)

    # Create a DF 
    df=pd.DataFrame(JobDict)

    # Replace line scape
    df['Job_Description'].replace(regex=True,inplace=True,to_replace=r'\n',value=r' ')

    # calculate Companies retrieved
    unique_companies_after=df.Company.nunique()
    
    return df

#### if main Section

In [ ]:
df_jobs=execution(function=runner_jobs,
                  working_dict=JobDict,
                  webdriver=driver,
                  dataSource=data_Source,
                  )

df_jobs.to_excel('jobResult.xlsx', index=False)